In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from PIL import Image
import os
import time
import glob
from PIL import *
import skimage


WIDTH = 100
HEIGHT = 100
SEQUENCE = np.array([])
BASIC_SEQUENCE = np.array([])
NEXT_SEQUENCE = np.array([])
NUMBER = 0

def image_initialize(image):
    picture = Image.open(image)
    picture = picture.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
    picture = picture.convert('L')
    picture.save('./temp.png')  
    data = np.array(picture.getdata()).reshape(WIDTH, HEIGHT, 1)
    return data

for file in glob.glob("../input/draper-satellite-image-chronology/train_sm/train_sm/set107_*.jpeg"):
    image_array = (image_initialize(file))/255
    SEQUENCE = np.append(SEQUENCE, image_array)
    NUMBER += 1
    print(SEQUENCE[0])
    print(NUMBER)
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH * HEIGHT)

np.savez('./sequence_array.npz', sequence_array=SEQUENCE)
print('Data saved.')
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

In [ ]:
def ssim_loss(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

In [ ]:
def psnr(y_true,y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv3D,ConvLSTM2D,BatchNormalization,MaxPooling2D, Dense, Flatten,GRU,Conv2D,Reshape,Permute,Lambda,Input
from tensorflow.keras.models import Sequential, load_model
from keras import backend as K
import pylab as pl
import time
# from tensorflow.keras.utils import multi_gpu_model
from keras import optimizers
import imutils
import cv2
# from google.colab.patches import cv2_imshow

def custom_layer(tensor):
    tensor=tensor[:,None]
    return tensor


#Loading array
SEQUENCE = np.load('./sequence_array.npz')['sequence_array']  # load array
n_samples = len(SEQUENCE)

WIDTH = 100
HEIGHT = 100
n_frames = 1

# step =1
SEQUENCE = SEQUENCE.reshape(n_samples, WIDTH, HEIGHT, 1)
BASIC_SEQUENCE = np.zeros((n_samples-n_frames, n_frames, WIDTH, HEIGHT, 1))
NEXT_SEQUENCE = np.zeros((n_samples-n_frames, n_frames, WIDTH, HEIGHT, 1))

for i in range(n_frames):
    BASIC_SEQUENCE[:, i, :, :, :] = SEQUENCE[i:i+n_samples-n_frames]
    NEXT_SEQUENCE[:, i, :, :, :] = SEQUENCE[i+1:i+n_samples-n_frames+1]





# model=Sequential()
# model.add(ConvLSTM2D(filters=256, kernel_size=(3, 3),input_shape=(None, WIDTH, HEIGHT, 1), padding="same", return_sequences=True))
# model.add(BatchNormalization())
# model.add(ConvLSTM2D(filters=256, kernel_size=(3, 3), padding="same", return_sequences=True))
# model.add(BatchNormalization())
# model.add(Conv3D(filters=1, kernel_size=(3, 3, 3), activation="relu", padding='same', data_format="channels_last"))
# model.compile(loss=ssim_loss, optimizer="adadelta",metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.RootMeanSquaredError(),ssim_loss,psnr])

model=Sequential()
model.add(Conv2D(filters=256, kernel_size=(3, 3),input_shape=(None, WIDTH, HEIGHT, 1), padding="same"))
model.add(Reshape((10000,256)))
model.add(GRU(256,return_sequences=True))
model.add(BatchNormalization())
model.add(Reshape((100,100,256)))
model.add(Lambda(custom_layer))
model.add(Conv3D(filters=1, kernel_size=(3, 3, 3), activation="relu", padding='same', data_format="channels_last"))
model.compile(loss=ssim_loss, optimizer="adadelta",metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.RootMeanSquaredError(),ssim_loss,psnr])

### Split data into training and test sets
# trainingfraction = 0.95
trainingfraction = 1.0
train_size = round(n_samples * trainingfraction)

### Train the network
# model.fit(BASIC_SEQUENCE[:train_size], NEXT_SEQUENCE[:train_size],  epochs=10, validation_split=0.05)
t_start = time.time()
H=model.fit(BASIC_SEQUENCE[:train_size], NEXT_SEQUENCE[:train_size], verbose=2, epochs=10)
t_end = time.time()

print('Training Elapsed time:')
print(t_end - t_start)

### Take an example from the test set and predict the next steps
index = 3

num_test_frames = 0 ### Number of frames to predict


train_pred = BASIC_SEQUENCE[index][:n_frames-num_test_frames:, ::, ::, ::]  ##(track)
for j in range(n_frames):
    t_start = time.time()
    new_pos = model.predict(train_pred[np.newaxis, :, :, :, :])
    t_end = time.time()
    print('Testing Elapsed time:')
    print(t_end - t_start)
    new = new_pos[:, -1, :, :, :]
    train_pred = np.concatenate((train_pred, new), axis=0)

### Compare predictions to the truth
truth = BASIC_SEQUENCE[index][:, :, :, :]

for i in range(n_frames):
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(121)
    ### In left panel show original then predicted frames
    # ax.text(1, 3, 'Predictions', fontsize=20, color='w')
    plt.text(1, 3, 'Predictions', fontsize=20)
    toplot = train_pred[i, ::, ::, 0]
    plt.imshow(toplot, cmap='binary')
    plt.savefig('%i_Prediction.png' % (i + 1))

for i in range(n_frames):   
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)
    toplot = truth[i, ::, ::, 0]
    plt.imshow(toplot, cmap='binary')
    plt.savefig('%i_GroundTruth.png' % (i + 1))

In [ ]:
!pip install imutils

In [ ]:
from skimage.metrics import structural_similarity as ssim
#Load Data
imageA = cv2.imread('./1_Prediction.png')
imageB = cv2.imread('./1_GroundTruth.png')

# convert the images to grayscale
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

# compute the Structural Similarity Index (SSIM) between the two
# images, ensuring that the difference image is returned
(score, diff) = ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))

